# Paper Grading Assistant

## Modeling

Data comes from this link:
- https://www.kaggle.com/c/asap-aes/data

Heavy inspiration drawn from:
- https://towardsdatascience.com/topic-modeling-articles-with-nmf-8c6b2a227a45

(Use incognito window when opening that link)

In [1]:
# !pip install gensim
import os, sys
from gensim import corpora, models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re


In [2]:
# Run the utilty functions from a seperate notebook
%run topic_model_utils.ipynb

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxw2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxw2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
data = pd.read_csv("D:\\Kaggle\\asap-aes\\training_set_rel3.tsv", sep='\t')
# data.head()

In [4]:
data['tokenized_essay'] = data.essay.apply(process_text)
data['max_score'] = 0

In [5]:
# replace NaN w/ 0
data = data.fillna(0)

# add a max_score column to use later 
# for standardizing scores
data['max_score'] = 0
data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,tokenized_essay,max_score
0,1,1,"Dear local newspaper, I think effects computer...",4,4,0.0,8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dear, local, newspaper, think, effect, comput...",0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,0.0,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dear, cap, believe, using, computer, benefit,...",0
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,0.0,7,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dear, cap, people, use, computer, agrees, ben...",0
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,0.0,10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dear, local, newspaper, cap, expert, computer...",0
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,0.0,8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dear, location, know, having, computer, posit...",0


In [6]:
# change max score col based on essay set
# max vals:
# set 1: 12
# set 2: 10 or 24, needs some experimenting
# set 3: 3
# set 4: 3
# set 5: 4
# set 6: 4
# set 7: 30
# set 8: 60

essay_sets = data.essay_set.unique()
for set_ in essay_sets:
    print(set_)
    if set_ == 1:
        data.loc[data.essay_set == set_, 'max_score'] = 12
    if set_ == 2:
        data.loc[data.essay_set == set_, 'max_score'] = 10
    if set_ == 3 or set_ == 4:
        data.loc[data.essay_set == set_, 'max_score'] = 3
    if set_ == 5 or set_ == 6:
        data.loc[data.essay_set == set_, 'max_score'] = 4
    if set_ == 7:
        data.loc[data.essay_set == set_, 'max_score'] = 30
    if set_ == 8:
        data.loc[data.essay_set == set_, 'max_score'] = 60
print(data.loc[data.essay_set == 1, 'max_score'])
print(data.loc[data.essay_set == 4, 'max_score'])
print(data.loc[data.essay_set == 7, 'max_score'])
print(data.loc[data.essay_set == 8, 'max_score'])

1
2
3
4
5
6
7
8
0       12
1       12
2       12
3       12
4       12
        ..
1778    12
1779    12
1780    12
1781    12
1782    12
Name: max_score, Length: 1783, dtype: int64
5309    3
5310    3
5311    3
5312    3
5313    3
       ..
7074    3
7075    3
7076    3
7077    3
7078    3
Name: max_score, Length: 1770, dtype: int64
10684    30
10685    30
10686    30
10687    30
10688    30
         ..
12248    30
12249    30
12250    30
12251    30
12252    30
Name: max_score, Length: 1569, dtype: int64
12253    60
12254    60
12255    60
12256    60
12257    60
         ..
12971    60
12972    60
12973    60
12974    60
12975    60
Name: max_score, Length: 723, dtype: int64


In [7]:
# create temp column for 
# models later internal classes
data['temp'] = 0
for set_ in essay_sets:
    if set_ == 2:
        data.loc[data.essay_set == set_, 'temp'] = data.loc[data.essay_set==set_,'domain1_score'] \
                                                   + data.loc[data.essay_set==set_,'domain2_score'] \
                                                   / data.loc[data.essay_set==set_,'max_score']
        continue
    else:
        data.loc[data.essay_set == set_, 'temp'] = data.loc[data.essay_set==set_,'domain1_score'] \
                                                   / data.loc[data.essay_set==set_,'max_score']

In [8]:
# re-classify each paper on a scale of 1-5,
# with 5 being a high score (like an A on an 
# ABCDF scale)
data['class'] = 1
for x in range(len(data)):
    if (data.temp[x]) >= .9:
        data['class'][x] = 5
        continue
    elif data.temp[x] >= .8 and data.temp[x] < .9:
        data['class'][x] = 4
        continue
    elif data.temp[x] >= .7 and data.temp[x] < .8:
        data['class'][x] = 3
        continue
    elif data.temp[x] >= .6 and data.temp[x] < .7:
        data['class'][x] = 2

<ipython-input-8-0a8b1fd4f84f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['class'][x] = 2
<ipython-input-8-0a8b1fd4f84f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['class'][x] = 3
<ipython-input-8-0a8b1fd4f84f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['class'][x] = 4
<ipython-input-8-0a8b1fd4f84f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# Initialize tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, 
                                   min_df=3, 
                                   max_features=no_features, 
                                   stop_words='english', 
                                   preprocessor=' '.join)
tfidf = tfidf_vectorizer.fit_transform(data['tokenized_essay'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Bag of words
tf_vectorizer = CountVectorizer(max_df=0.85, 
                                min_df=3, 
                                max_features=no_features, 
                                stop_words='english', 
                                preprocessor=' '.join)
tf = tf_vectorizer.fit_transform(data['tokenized_essay'])
tf_feature_names = tf_vectorizer.get_feature_names()

# Word2Vec
word2vec = WordEmbeddingsService()
word2vec_model = word2vec.train_w2v_model(tokenized_text=data['tokenized_essay'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxw2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maxw2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# create a few different vecotrizations of the data
# to see which version does the best

X_tfidf = tfidf
X_tf = tf
X_w2v = word2vec.create_word_embeddings(data['tokenized_essay'], word2vec_model)
y = data['class']

In [12]:
# import all the different classifiers 
# to test with the paper scores
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

from xgboost import XGBClassifier

In [13]:
def make_classification(classifier, X, y, rs=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = rs)
    # try:
    #     classifier.fit(X_train, y_train)
    # except:
    #     X_train = np.array(X_train)
    #     X_test = np.array(X_test)
    #     y_train = np.array(y_train)
    #     y_test = np.array(y_test)
    #     classifier.fit(X_train, y_train)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score, prec_score, rec_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return cm, acc_score, f1, prec_score, rec_score

def make_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    acc_score = accuracy_score(y_test, y_pred)
    prec_score = precision_score(y_test, y_pred, average='weighted')
    rec_score = recall_score(y_test, y_pred, average='weighted')
    return cm, acc_score, prec_score, rec_score

In [14]:
# create a dictionary of all the different classifiers
# to loop through.
# There are some models for unsupervised just for comparison.
classifiers = {
    "knn": KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2),
    "nb" : MultinomialNB(), 
    "log_reg": LogisticRegression(random_state=0),
    "lin_svm" : SVC(kernel = 'linear', random_state = 0), # took too long with word2vec (more than 5000 secs)
    "rbf_svm" : SVC(kernel = 'rbf', random_state = 0),
    "tree" : DecisionTreeClassifier(criterion = 'entropy', random_state = 0),
    "rf" : RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0),
    "ada" : AdaBoostClassifier(random_state = 0),
    "gb" : GradientBoostingClassifier(random_state = 0),
    "xgb" : XGBClassifier(random_state = 0),
}

In [15]:
# tfidf vectors first, 3 min
tfidf_res = {}
for key in classifiers.keys():
    print(key)
    cm, acc, f1, prec, rec = make_classification(classifiers[key], X_tfidf, y)
    tfidf_res[key] = {
        'cm' : cm,
        'acc' : acc,
        'f1' : f1,
        'prec' : prec,
        'rec' : rec
    }
    print("==============")

knn
nb
log_reg


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lin_svm
rbf_svm
tree
rf
ada
gb
xgb


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:45:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [16]:
# repeat classification with bag of words models, 2.5 min
tf_res = {}
for key in classifiers.keys():
    print(key)
    cm, acc, f1, prec, rec = make_classification(classifiers[key], X_tf, y)
    print("==============")
    tf_res[key] = {
        'cm' : cm,
        'acc' : acc,
        'f1' : f1,
        'prec' : prec,
        'rec' : rec
    }

knn


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


nb
log_reg


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lin_svm
rbf_svm
tree
rf
ada
gb
xgb
[15:48:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [21]:
# repeat classification with word2vec models, 5 min
w2v_res = {}
for key in classifiers.keys():
    # lin_svm takes more than 1 hour on its own.
    # nb doesn't accept negative numbers from the vectors.
    if key == 'lin_svm' or key == 'nb': 
        continue
    print(key)
    try:
        cm, acc, f1, prec, rec = make_classification(classifiers[key], X_w2v, y)
    except:
        cm, acc, f1, prec, rec = 0,0,0,0,0
    print("==============")
    w2v_res[key] = {
        'cm' : cm,
        'acc' : acc,
        'f1' : f1,
        'prec' : prec,
        'rec' : rec
    }

knn
nb
log_reg


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


rbf_svm


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tree
rf
ada
gb
xgb


C:\Users\maxw2\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:55:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [22]:
# everything else being equal,
# we want the one with highest precisions 
# (precision is affected by FP, which would be 
# overestimation of the grade of the paper)

for key in classifiers.keys():
    try:
        print(key)
        print("==================")
        print("tfidf acc: ", tfidf_res[key]['acc'])
        print("tfidf f1: ", tfidf_res[key]['f1'])
        print("tfidf precision: ", tfidf_res[key]['prec'])
        print("tfidf recall: ", tfidf_res[key]['rec'])
        print("==================")
        print("tf acc: ", tf_res[key]['acc'])
        print("tf f1: ", tf_res[key]['f1'])
        print("tf precision: ", tf_res[key]['prec'])
        print("tf recall: ", tf_res[key]['rec'])
        print("==================")
        print("w2v acc: ", w2v_res[key]['acc'])
        print("w2v f1: ", w2v_res[key]['f1'])
        print("w2v precision: ", w2v_res[key]['prec'])
        print("w2v recall: ", w2v_res[key]['rec'])
        print("==================")
    except:
        pass

knn
tfidf acc:  0.5597072419106317
tfidf f1:  0.5180089403445932
tfidf precision:  0.5780338452801514
tfidf recall:  0.5597072419106317
tf acc:  0.5050077041602465
tf f1:  0.42539275142673827
tf precision:  0.5511446307730005
tf recall:  0.5050077041602465
w2v acc:  0.5057781201848999
w2v f1:  0.5024756948003423
w2v precision:  0.5067483884578161
w2v recall:  0.5057781201848999
nb
tfidf acc:  0.573959938366718
tfidf f1:  0.589093281704411
tfidf precision:  0.6358654965846269
tfidf recall:  0.573959938366718
tf acc:  0.5450693374422187
tf f1:  0.5756543060296143
tf precision:  0.655096792659111
tf recall:  0.5450693374422187
w2v acc:  0
w2v f1:  0
w2v precision:  0
w2v recall:  0
log_reg
tfidf acc:  0.6879815100154083
tfidf f1:  0.6826731058258083
tfidf precision:  0.6840742515662983
tfidf recall:  0.6879815100154083
tf acc:  0.6687211093990755
tf f1:  0.6659915493079803
tf precision:  0.6647184483147467
tf recall:  0.6687211093990755
w2v acc:  0.515793528505393
w2v f1:  0.4996253017823

Here are the best results from the training above.

### log_reg
- tfidf acc:  0.6879815100154083
- tfidf f1:  0.6826731058258083
- tfidf precision:  0.6840742515662983
- tfidf recall:  0.6879815100154083

### lin_svm
- tfidf acc:  0.6798921417565486
- tfidf f1:  0.6796612464208107
- tfidf precision:  0.6851511607249138
- tfidf recall:  0.6798921417565486

### rbf_svm
- tf acc:  0.714175654853621
- tf f1:  0.7137245090277426
- tf precision:  0.7245950249260565
- tf recall:  0.714175654853621

### tree
- tfidf acc:  0.6302003081664098
- tfidf f1:  0.6289259312408813
- tfidf precision:  0.6278195501435767
- tfidf recall:  0.6302003081664098

### rf
- tfidf acc:  0.687211093990755
- tfidf f1:  0.6795152273421196
- tfidf precision:  0.6789466187123883
- tfidf recall:  0.687211093990755

### ada
- tf acc:  0.5670261941448382
- tf f1:  0.5388070054332165
- tf precision:  0.5626726131804644
- tf recall:  0.5670261941448382


### gb
- tfidf acc:  0.7126348228043143
- tfidf f1:  0.7112178825280995
- tfidf precision:  0.7124145728645078
- tfidf recall:  0.7126348228043143

### These were the best of the word2vec models (Gradient boosting)
- w2v acc:  0.6771956856702619
- w2v f1:  0.6786444062419466
- w2v precision:  0.6813704223813889
- w2v recall:  0.6771956856702619

### xgb
- tfidf acc:  0.7245762711864406
- tfidf f1:  0.7223852160700287
- tfidf precision:  0.7212926330136676
- tfidf recall:  0.7245762711864406




In [23]:
for key in classifiers.keys():
    try:
        print(key)
        print("==================")
        print("tfidf cm: \n", tfidf_res[key]['cm'])
        print("==================")
        print("tf cm: \n", tf_res[key]['cm'])
        print("==================")
        print("w2v cm: \n", w2v_res[key]['cm'])
    except:
        pass

knn
tfidf acc: 
 [[853  53  28   0   3]
 [405  88   7   5  13]
 [252  22  91   6  37]
 [ 62  13   2   5   0]
 [161  27  45   2 416]]
tf acc: 
 [[929   6   2   0   0]
 [498  18   0   0   2]
 [343   5  54   0   6]
 [ 77   5   0   0   0]
 [270  16  55   0 310]]
w2v acc: 
 [[568 168 133   7  61]
 [240 190  23  24  41]
 [192  44 133   7  32]
 [ 27  32  17   5   1]
 [123  58  52   1 417]]
nb
tfidf acc: 
 [[590 182 138  17  10]
 [161 281   0  73   3]
 [124  42 197  45   0]
 [ 19  13   0  49   1]
 [ 58  97  95  28 373]]
tf acc: 
 [[499 215 164  54   5]
 [103 259   0 156   0]
 [ 79  31 229  69   0]
 [ 10  16   0  56   0]
 [ 34 106 107  32 372]]
w2v acc: 
 0
log_reg
tfidf acc: 
 [[788  81  53   1  14]
 [151 312   7   6  42]
 [ 90  72 203   6  37]
 [  8  39  12  14   9]
 [ 22  59  88  13 469]]
tf acc: 
 [[752  94  72   5  14]
 [145 261  35  22  55]
 [ 89  49 209  13  48]
 [  6  24  23  19  10]
 [ 19  50  76  11 495]]
w2v acc: 
 [[589 205  84   0  59]
 [246 208   0   0  64]
 [168  38 115   0  87]


In [24]:
# check punctuation
def check_punctuation(raw_text, punctuation):
    article = raw_text.split(str(punctuation))
    for a in article:
        a = a.strip()
    sentences = []
    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        sentences.pop() 
    
    return sentences